# Create Kaggle Secret

In this Notebook you will create a Kubernetes secret from a Kaggle API key. The main purpose of this Notebook is to facilitate the seamless integration of Kaggle CLI operations within a Kubeflow pipeline. Here's a breakdown of the steps involved:

1. **Read the Kaggle API Key**: The Notebook begins by reading the `kaggle.json` file located in the `.kaggle` directory. This file contains the necessary API credentials for accessing
   the Kaggle platform.
   
   > **❗Note**: You should already have an API token. If you don't follow the instructions in the [docs](https://www.kaggle.com/docs/api#authentication).

1. **Extract API Credentials**: Once the `kaggle.json` file is read, its contents are extracted. These contents include the username and API key required for authenticating with the
   Kaggle API.

1. **Create a Kubernetes Secret**: With the extracted credentials, the Notebook proceeds to create a Kubernetes secret. This secret will securely store the Kaggle API key within the
   Kubernetes cluster.

1. **Use the Secret in Kubeflow Pipelines**: The Kubernetes secret can then be referenced in Kubeflow pipelines steps to perform various actions using the Kaggle CLI, such as downloading
   datasets, submitting competitions, and more.

By automating these steps, this Notebook streamlines the workflow for users who need to interact with Kaggle as part of their data science and machine learning projects within a Kubeflow environment. This ensures that the API credentials are securely managed and easily accessible for subsequent tasks.

In [ ]:
import json
import base64

from pathlib import Path
from kubernetes import client
from kubernetes.config.incluster_config import InClusterConfigLoader
from kubernetes.client import V1Secret, V1ObjectMeta

Start by instantiating the Kubernetes Python client. You will need this to submit the Secret you will create.

In [ ]:
root = Path("/")
sa = root/Path("var/run/secrets/kubernetes.io/serviceaccount")
ns = open(sa/"namespace", "r").read()

# load and set kubernetes config
loader = InClusterConfigLoader(
    token_filename=sa/"token",
    cert_filename=sa/"ca.crt",
)
loader.load_and_set()

v1 = client.CoreV1Api()

Next, get the Kaggle credentials. We assume that there is a `kaggle.json` file under `~/.kaggle`. If not, see the [docs](https://www.kaggle.com/docs/api#authentication) on how to create one. To complete this step, you should have an account with Kaggle.

In [ ]:
with open(Path.home()/".kaggle"/"kaggle.json", "r") as f:
    kaggle_creds = json.load(f)

Finally, create the Kubernetes Secret and submit it using the client. We will create a Secret of type `Opaque` for this. For a complete list of Kubernetes secret types, see thee [docs](https://kubernetes.io/docs/concepts/configuration/secret/#secret-types).

In [ ]:
kaggle_secret = V1Secret(
    api_version="v1",
    kind="Secret",
    metadata=V1ObjectMeta(name="kaggle-secret"),
    string_data=kaggle_creds
)

In [ ]:
v1.create_namespaced_secret(namespace=ns, body=kaggle_secret)